In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.tsa.stattools import adfuller,  kpss
from scipy import stats
from operator import index
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import roc_auc_score, f1_score
from statsmodels.tsa.stattools import pacf
from scipy.signal import argrelextrema
from lightgbm import LGBMClassifier
from joblib import load

In [2]:
test = pd.read_parquet('test.parquet')

test = pd.DataFrame(test)

idd = test['id']

test = test.drop('id', axis=1)

In [3]:
def is_stationarity_adf(ts):
    if adfuller(ts)[1] < 0.05:
        return True
    return False

def is_stationarity_kpss(ts):
    if kpss(ts)[1] < 0.05:
        return False
    return True

In [4]:
def diff_ts(df):
    
    for idx, row in df.iterrows():
        
        temp_df = pd.DataFrame({'values' : row['values']}, index=row['dates'])

        temp_df = temp_df.dropna()

        if not temp_df.empty:

          if not (is_stationarity_adf(temp_df['values']) and is_stationarity_kpss(temp_df['values'])):


            diff_values = temp_df['values'].diff().diff().dropna()

            if not diff_values.empty:


              df.at[idx, 'values'] = diff_values.values

              df.at[idx, 'dates'] = diff_values.index



In [5]:
def inser_features_and_create_data(df):
    
    features = []

    for idx, row in df.iterrows():
        values = row['values']

        feature_row = {
            'mean': np.mean(values),
            'std': np.std(values),
            'min': np.min(values),
            'max': np.max(values),
            'median': np.median(values),
            'autocorr': pd.Series(values).autocorr(lag=1),
            'skewness': pd.Series(values).skew(),
            'kurtosis': pd.Series(values).kurt(),
            'sign_changes': np.sum(np.diff(np.sign(values)) != 0),
            'autocorr_lag_2': pd.Series(values).autocorr(lag=2),
            'pacf_lag_1': pacf(values, nlags=5)[1],
            'num_peaks': len(argrelextrema(np.array(values), np.greater)[0]),
            'num_troughs': len(argrelextrema(np.array(values), np.less)[0])
        }
        features.append(feature_row)

    features_df = pd.DataFrame(features)
    features_df = features_df.apply(lambda x: x.fillna(x.median()))
    
    features_df = features_df.apply(pd.to_numeric, errors='coerce')
    
    return features_df

In [6]:
diff_ts(test)

test = inser_features_and_create_data(test)

/home/fltww/anaconda3/envs/ML/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:2023: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  warn_msg.format(direction="greater"), InterpolationWarning
/home/fltww/anaconda3/envs/ML/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:2023: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  warn_msg.format(direction="greater"), InterpolationWarning
/home/fltww/anaconda3/envs/ML/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:2023: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  warn_msg.format(direction="greater"), InterpolationWarning
/home/fltww/anaconda3/envs/ML/lib/pyt

/home/fltww/anaconda3/envs/ML/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:2023: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  warn_msg.format(direction="greater"), InterpolationWarning
/home/fltww/anaconda3/envs/ML/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:2023: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  warn_msg.format(direction="greater"), InterpolationWarning
/home/fltww/anaconda3/envs/ML/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:2019: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  warn_msg.format(direction="smaller"), InterpolationWarning
/home/fltww/anaconda3/envs/ML/lib/pyt

/home/fltww/anaconda3/envs/ML/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:2023: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  warn_msg.format(direction="greater"), InterpolationWarning
/home/fltww/anaconda3/envs/ML/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:2023: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  warn_msg.format(direction="greater"), InterpolationWarning
/home/fltww/anaconda3/envs/ML/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:2023: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  warn_msg.format(direction="greater"), InterpolationWarning
/home/fltww/anaconda3/envs/ML/lib/pyt

/home/fltww/anaconda3/envs/ML/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:2023: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  warn_msg.format(direction="greater"), InterpolationWarning
/home/fltww/anaconda3/envs/ML/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:2023: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  warn_msg.format(direction="greater"), InterpolationWarning
/home/fltww/anaconda3/envs/ML/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:2023: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  warn_msg.format(direction="greater"), InterpolationWarning
/home/fltww/anaconda3/envs/ML/lib/pyt

/home/fltww/anaconda3/envs/ML/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:2023: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  warn_msg.format(direction="greater"), InterpolationWarning
/home/fltww/anaconda3/envs/ML/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:2023: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  warn_msg.format(direction="greater"), InterpolationWarning
/home/fltww/anaconda3/envs/ML/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:2023: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  warn_msg.format(direction="greater"), InterpolationWarning
/home/fltww/anaconda3/envs/ML/lib/pyt

...

/home/fltww/anaconda3/envs/ML/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:2023: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  warn_msg.format(direction="greater"), InterpolationWarning
/home/fltww/anaconda3/envs/ML/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:2023: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  warn_msg.format(direction="greater"), InterpolationWarning
/home/fltww/anaconda3/envs/ML/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:2023: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  warn_msg.format(direction="greater"), InterpolationWarning
/home/fltww/anaconda3/envs/ML/lib/pyt

/home/fltww/anaconda3/envs/ML/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:2023: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  warn_msg.format(direction="greater"), InterpolationWarning
/home/fltww/anaconda3/envs/ML/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:2023: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  warn_msg.format(direction="greater"), InterpolationWarning
/home/fltww/anaconda3/envs/ML/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:2023: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  warn_msg.format(direction="greater"), InterpolationWarning
/home/fltww/anaconda3/envs/ML/lib/pyt

/home/fltww/anaconda3/envs/ML/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:2023: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  warn_msg.format(direction="greater"), InterpolationWarning
/home/fltww/anaconda3/envs/ML/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:2023: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  warn_msg.format(direction="greater"), InterpolationWarning
/home/fltww/anaconda3/envs/ML/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:2019: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  warn_msg.format(direction="smaller"), InterpolationWarning
/home/fltww/anaconda3/envs/ML/lib/pyt

/home/fltww/anaconda3/envs/ML/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:2023: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  warn_msg.format(direction="greater"), InterpolationWarning
/home/fltww/anaconda3/envs/ML/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:2023: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  warn_msg.format(direction="greater"), InterpolationWarning
/home/fltww/anaconda3/envs/ML/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:2023: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  warn_msg.format(direction="greater"), InterpolationWarning
/home/fltww/anaconda3/envs/ML/lib/pyt

/home/fltww/anaconda3/envs/ML/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:2023: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  warn_msg.format(direction="greater"), InterpolationWarning
/home/fltww/anaconda3/envs/ML/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:2023: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  warn_msg.format(direction="greater"), InterpolationWarning
/home/fltww/anaconda3/envs/ML/lib/python3.7/site-packages/statsmodels/tsa/stattools.py:2023: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  warn_msg.format(direction="greater"), InterpolationWarning
/home/fltww/anaconda3/envs/ML/lib/pyt

In [7]:
test.head()

,mean,std,min,max,median,autocorr,skewness,kurtosis,sign_changes,autocorr_lag_2,pacf_lag_1,num_peaks,num_troughs
0,0.037895,1.505985,-3.14,3.13,-0.01,0.361637,0.067832,-0.342699,21,0.301715,0.363091,16,17
1,-0.016909,1.249663,-2.24,3.85,-0.08,0.013262,0.899283,1.058050,23,0.182465,0.012821,19,19
2,0.043774,1.989945,-4.98,3.96,0.06,-0.802971,-0.338582,-0.182292,38,0.474984,-0.798770,19,20
3,0.159231,1.313112,-2.73,3.43,0.11,0.260440,0.392145,0.022481,38,0.279004,0.260612,26,26
4,-0.000635,2.342460,-4.65,6.14,0.17,-0.607783,0.201127,-0.507019,46,-0.014164,-0.612795,23,22


In [8]:
model = load('model.joblib')

In [9]:
predict = model.predict_proba(test)[:,1]

res = pd.DataFrame({'id' : idd, 'score' : predict})

In [10]:
res

,id,score
0,6125,0.255541
1,26781,0.155929
2,13333,0.000112
3,53218,0.053970
4,84204,0.000079
...,...,...
19995,80341,0.370286
19996,5891,0.095412
19997,29091,0.000133
19998,85877,0.000110


In [11]:
res.to_csv('submission.csv', index=False)